### importing essential libraries

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

### main page url

In [2]:
url = "https://www.booking.com/city/nl/amsterdam.html?aid=306395&label=nl-kuqoEcU6c_NqQLhvY8YAGwS151853408337%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-298354739691%3Alp9076916%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9YZVcNNsENnH02-pWD53qm9c&sid=78edbdc975c2bacefb3bd3150d9c346c"

In [3]:
page = requests.get(url)

In [4]:
page

<Response [200]>

In [5]:
soup = BeautifulSoup(page.text,'html')

### extracting every hotel hyperlink

In [6]:
table = soup.find('div', attrs = {"data-et-view" : "OATBaQJbVTbZXWSYHCSdUWAUC:1"})

In [7]:
every_hotel = table.find_all('a',attrs = {"data-et-click" : "customGoal:OATBaQJbVTbZXWSYHCSdUWAUC:2"})

In [8]:
every_hotel_link = [('https://www.booking.com' + x.get('href')) for x in every_hotel]

In [9]:
every_hotel_link

['https://www.booking.com/hotel/nl/park-inn-by-radisson-amsterdam-city-west.html',
 'https://www.booking.com/hotel/nl/room-mate-aitana.html',
 'https://www.booking.com/hotel/nl/nhghkrasnapolsky.html',
 'https://www.booking.com/hotel/nl/radissonamsterdam.html',
 'https://www.booking.com/hotel/nl/victoria.html',
 'https://www.booking.com/hotel/nl/ibiscentre.html',
 'https://www.booking.com/hotel/nl/eden-hotel-amsterdam.html',
 'https://www.booking.com/hotel/nl/jollycarlton.html',
 'https://www.booking.com/hotel/nl/joy.html',
 'https://www.booking.com/hotel/nl/holiday-inn-express-amsterdam-arena-towers.html']

### scraping every hotel data, which includes it's name, address, and facilities

In [10]:
data = []
count = 1
for x in every_hotel_link:
    missing = False # will check and help in dropping the record which have missing values
    
    hotel_data = [] # contain the data of current hotel in iteration
    
    url = x
    page = requests.get(url)
    print(page)
    soup = BeautifulSoup(page.text,'html')
    
    hotel_data.append(soup.find('h2',class_ = 'd2fee87262 pp-header__title').text.strip()) # hotel name
    
    x = soup.find('p',class_ = 'address address_clean').text.strip() # hotel address
    hotel_data.append(x.split('\n')[0])
    
    x = soup.find('ul',class_ = "aca0ade214 aaf30230d9 c2931f4182 e7d9f93f4d ed5cdd3fb3 fb5b81c565") # extracting regular facilities
    if x is not None: 
        y = [a.text for a in x.find_all('div',class_ = "aca0ade214 aaf30230d9 cd2e7d62b0")]
        hotel_data.append(','.join(y))
    else : 
        print(f'{count} regular facilities not found')
        missing = True
    
    x = soup.find('ul',class_ = 'c807d72881 d1a624a1cc e10711a42e') # extracting popular facilities
    if x is not None:
        y = [a.text for a in x.find_all('span',class_ = 'a5a5a75131')]
        hotel_data.append(','.join(y))
    else  :
        print(f'{count} popular facilities not found')
        missing = True
        
    if missing == False :
        data.append(hotel_data)
        
        
    count += 1
    print('')

<Response [200]>

<Response [200]>

<Response [200]>

<Response [200]>

<Response [200]>

<Response [200]>

<Response [200]>

<Response [200]>

<Response [200]>
9 regular facilities not found

<Response [200]>



In [11]:
data

[['Park Inn by Radisson Amsterdam City West',
  'La Guardiaweg 59, Westpoort, 1043 DE Amsterdam, Netherlands',
  'City view,Garden,Free WiFi,Bathtub,Air conditioning,24-hour front desk,Key card access,Daily housekeeping,Non-smoking rooms,Baggage storage',
  'Private Parking,Free WiFi,Family rooms,Non-smoking rooms,Fitness center,Restaurant,Facilities for disabled guests,Tea/Coffee Maker in All Rooms,Bar,Very Good Breakfast'],
 ['Room Mate Aitana',
  'IJdock 6, Amsterdam City Center, 1013 MM Amsterdam, Netherlands',
  'City view,Free WiFi,Bathtub,Air conditioning,24-hour front desk,Key card access,Daily housekeeping,Non-smoking rooms,Safe,Baggage storage',
  'Parking on site,Free WiFi,Family rooms,Non-smoking rooms,Fitness center,Restaurant,Room service,Tea/Coffee Maker in All Rooms,Bar,Very Good Breakfast'],
 ['Anantara Grand Hotel Krasnapolsky Amsterdam',
  'Dam 9, Amsterdam City Center, 1012 JS Amsterdam, Netherlands',
  'City view,Pet friendly,Free WiFi,Terrace,Balcony,Air condition

### converting data into numpy array

In [12]:
data = np.array(data)

In [13]:
data.shape

(9, 4)

### converting array into dataframe

In [14]:
df = pd.DataFrame(data,columns = ['Hotel_Name','Hotel_Address','Regular_Facilities','Popular_Facilities'])

In [15]:
df

,Hotel_Name,Hotel_Address,Regular_Facilities,Popular_Facilities
0,Park Inn by Radisson Amsterdam City West,"La Guardiaweg 59, Westpoort, 1043 DE Amsterdam...","City view,Garden,Free WiFi,Bathtub,Air conditi...","Private Parking,Free WiFi,Family rooms,Non-smo..."
1,Room Mate Aitana,"IJdock 6, Amsterdam City Center, 1013 MM Amste...","City view,Free WiFi,Bathtub,Air conditioning,2...","Parking on site,Free WiFi,Family rooms,Non-smo..."
2,Anantara Grand Hotel Krasnapolsky Amsterdam,"Dam 9, Amsterdam City Center, 1012 JS Amsterda...","City view,Pet friendly,Free WiFi,Terrace,Balco...","Private Parking,Free WiFi,Spa,Family rooms,Non..."
3,"Radisson Blu Hotel, Amsterdam City Center","Rusland 17, Amsterdam City Center, 1012 CK Ams...","City view,Free WiFi,Terrace,Balcony,Bathtub,Ai...","Private Parking,Free WiFi,Family rooms,Non-smo..."
4,Park Plaza Victoria Amsterdam,"Damrak 1 - 5, Amsterdam City Center, 1012 LG A...","City view,Indoor swimming pool,Free WiFi,Batht...","Indoor swimming pool,Parking,Free WiFi,Family ..."
5,ibis Amsterdam Centre,"Stationsplein 49, Amsterdam City Center, 1012 ...","River view,Pet friendly,Free WiFi,Private Bath...","Free WiFi,Family rooms,Non-smoking rooms,Resta..."
6,Eden Hotel Amsterdam,"Amstel 144, Amsterdam City Center, 1017 DA Ams...","City view,Free WiFi,Bathtub,Air conditioning,2...","Parking,Free WiFi,Family rooms,Non-smoking roo..."
7,NH Collection Amsterdam Flower Market,"Vijzelstraat 4, Amsterdam City Center, 1017 HK...","City view,Pet friendly,Free WiFi,Balcony,Batht...","Free WiFi,Family rooms,Non-smoking rooms,Fitne..."
8,"Holiday Inn Express Amsterdam Arena Towers, an...","Hoogoorddreef 66, Zuidoost, 1101 BE Amsterdam,...","View,Free WiFi,Air conditioning,Private Bathro...","Private Parking,Free WiFi,Non-smoking rooms,Re..."


### saving dataframe as csv file

In [17]:
df.to_csv('hotel_scrap.csv',index = False)